In [1]:
import keyring
from pprint import pprint

api_key_irp = keyring.get_password("irp_api_key", "birdblues")
api_secret_irp = keyring.get_password("irp_api_secret", "birdblues")

api_key_isa = keyring.get_password("isa_api_key", "birdblues")
api_secret_isa = keyring.get_password("isa_api_secret", "birdblues")

api_key_0 = keyring.get_password("api_key_0", "birdblues")
api_secret_0 = keyring.get_password("api_secret_0","birdblues")

api_key_1 = keyring.get_password("api_key_1", "birdblues")
api_secret_1 = keyring.get_password("api_secret_1","birdblues")

api_key_2 = keyring.get_password("api_key_2", "birdblues")
api_secret_2 = keyring.get_password("api_secret_2","birdblues")

# print(f"api_key_irp = {api_key_irp}")
# print(f"api_secret_irp = {api_secret_irp}")
# print(f"api_key_isa = {api_key_isa}")
# print(f"api_secret_isa = {api_secret_isa}")
# print(f"api_key_0 = {api_key_0}")
# print(f"api_secret_0 = {api_secret_0}")
# print(f"api_key_1 = {api_key_1}")
# print(f"api_secret_1 = {api_secret_1}")
# print(f"api_key_2 = {api_key_2}")
# print(f"api_secret_2 = {api_secret_2}")

In [2]:
acc_no_0 = "63981494-01" # 위탁
acc_no_1 = "64267316-22" # 연금저축 1
acc_no_2 = "64012548-22" # 연금저축 2
acc_no_isa = "64043278-01" # ISA
acc_no_irp = "64012548-29" # IRP

In [3]:
import keyring

token_0 = keyring.get_password("access_token_0", "birdblues")
token_1 = keyring.get_password("access_token_1", "birdblues")
token_2 = keyring.get_password("access_token_2", "birdblues")
token_3 = keyring.get_password("access_token_3", "birdblues")
token_4 = keyring.get_password("access_token_4", "birdblues")

# print(f"token_0 = {token_0}")
# print(f"token_1 = {token_1}")
# print(f"token_2 = {token_2}")
# print(f"token_3 = {token_3}")
# print(f"token_4 = {token_4}")

In [4]:
import requests
import json

def get_balance(access_token, app_key, app_secret, acc_no):
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/inquire-balance"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC8434R"
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "AFHR_FLPR_YN": "N",  # 시간외단일가여부
        "OFL_YN": "",  # 공란
        "INQR_DVSN": "01",  # 조회구분
        "UNPR_DVSN": "01",  # 단가구분
        "FUND_STTL_ICLD_YN": "N",  # 펀드결제분포함여부
        "FNCG_AMT_AUTO_RDPT_YN": "N",  # 융자금액자동상환여부        
        "PRCS_DVSN": "00",  # 처리구분(00: 전일매매포함)
        "CTX_AREA_FK100": "",  # 연속조회검색조건
        "CTX_AREA_NK100": ""  # 연속조회키
    }
    res = requests.get(url, headers=headers, params=params)
    return res.json()
    

In [5]:
def get_balance_irp(access_token, app_key, app_secret, acc_no):
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/pension/inquire-balance"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC2208R"
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "ACCA_DVSN_CD":"00",
        "INQR_DVSN":"00",
        "CTX_AREA_FK100": "",  # 연속조회검색조건
        "CTX_AREA_NK100": ""  # 연속조회키
    }
    res = requests.get(url, headers=headers, params=params)
    return res.json()

In [6]:
json_0 = get_balance(token_0, api_key_0, api_secret_0, acc_no_0)
json_1 = get_balance(token_1, api_key_1, api_secret_1, acc_no_1)
json_2 = get_balance(token_2, api_key_2, api_secret_2, acc_no_2)
json_3 = get_balance(token_3, api_key_isa, api_secret_isa, acc_no_isa)
json_4 = get_balance_irp(token_4, api_key_irp, api_secret_irp, acc_no_irp)
json_4

{'rt_cd': '1', 'msg1': '기간이 만료된 token 입니다.', 'msg_cd': 'EGW00123'}

In [7]:
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

def get_balance_json(json):
    cash = json['output2'][0]['prvs_rcdl_excc_amt']
    ap = pd.DataFrame.from_records(json['output1'])
    if(ap.size > 0):
        ap = ap[["pdno", "prdt_name", "evlu_amt", "prpr", "hldg_qty", "pchs_amt", "pchs_avg_pric", "evlu_pfls_rt"]]
        ap.columns = ["코드", "이름", "평가금액", "단가", "수량", "매입금액", "매입단가", "수익률"]
        new_row = {"코드": "0", "이름": "현금", "평가금액": cash, "단가": 1, "수량": cash, "매입금액": cash, "매입단가": 1, "수익률": 0}
        new_row_df = pd.DataFrame([new_row])

        # concat을 사용하여 새로운 행 추가
        ap = pd.concat([ap, new_row_df], ignore_index=True)
        
        # 데이터 타입 변환
        ap["평가금액"] = pd.to_numeric(ap["평가금액"], errors='coerce')
        ap["매입금액"] = pd.to_numeric(ap["매입금액"], errors='coerce')
        ap["단가"] = pd.to_numeric(ap["단가"], errors='coerce')
        ap["매입단가"] = pd.to_numeric(ap["매입단가"], errors='coerce')
        ap["수량"] = pd.to_numeric(ap["수량"], errors='coerce')
        ap["수익률"] = pd.to_numeric(ap["수익률"], errors='coerce')
        ap["계좌"] = json['ctx_area_fk100'][:8] + '-' + json['ctx_area_fk100'][9:11]
    return ap

In [8]:
import pandas as pd


def get_balance_json_irp(json):
    cash = json['output2']['prvs_rcdl_excc_amt']
    ap = pd.DataFrame.from_records(json['output1'])
    if(ap.size > 0):
        ap = ap[["pdno", "prdt_name", "evlu_amt", "prpr", "hldg_qty", "pchs_amt", "pchs_avg_pric", "evlu_erng_rt"]]
        ap.columns = ["코드", "이름", "평가금액", "단가", "수량", "매입금액", "매입단가", "수익률"]
        new_row = {"코드": "0", "이름": "현금", "평가금액": cash, "단가": 1, "수량": cash, "매입금액": cash, "매입단가": 1, "수익률": 0}
        new_row_df = pd.DataFrame([new_row])

        # concat을 사용하여 새로운 행 추가
        ap = pd.concat([ap, new_row_df], ignore_index=True)

        # 데이터 타입 변환
        ap["평가금액"] = pd.to_numeric(ap["평가금액"], errors='coerce')
        ap["매입금액"] = pd.to_numeric(ap["매입금액"], errors='coerce')
        ap["단가"] = pd.to_numeric(ap["단가"], errors='coerce')
        ap["매입단가"] = pd.to_numeric(ap["매입단가"], errors='coerce')
        ap["수량"] = pd.to_numeric(ap["수량"], errors='coerce')
        ap["수익률"] = pd.to_numeric(ap["수익률"], errors='coerce')
        ap["계좌"] = json['ctx_area_fk100'][:8] + '-' + json['ctx_area_fk100'][9:11]
    return ap

In [9]:
df_0 = get_balance_json(json_0)
df_1 = get_balance_json(json_1)
df_2 = get_balance_json(json_2)
df_3 = get_balance_json(json_3)
df_4 = get_balance_json_irp(json_4)
df = pd.concat([df_0, df_1, df_2, df_3, df_4], ignore_index=True)
# df_0 = df_0.sort_values(by='이름')
df_1 = df_1.sort_values(by='이름')
df_2 = df_2.sort_values(by='이름')
df_3 = df_3.sort_values(by='이름')
df_4 = df_4.sort_values(by='이름')

total_1 = df_1["평가금액"].sum()
total_2 = df_2["평가금액"].sum()
total_3 = df_3["평가금액"].sum()
total_4 = df_4["평가금액"].sum()

KeyError: 'output2'

In [ ]:
df_1["비중"] = df_1["평가금액"] / total_1 * 100
df_1

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌,비중
7,411060,ACE KRX금현물,5145000,17150,300,4933313,16444.38,4.29,64267316-22,9.97
6,365780,ACE 국고채10년,4264200,92700,46,4213150,91590.22,1.21,64267316-22,8.26
5,360200,ACE 미국S&P500,5169500,21100,245,5112805,20868.59,1.10,64267316-22,10.02
8,456880,ACE 미국달러SOFR금리(합성),4393010,11470,383,4365560,11398.33,0.62,64267316-22,8.51
3,308620,KODEX 미국10년국채선물,4914160,11590,424,4875895,11499.75,0.78,64267316-22,9.52
1,283580,KODEX 차이나CSI300,1869560,12140,154,1944460,12626.36,-3.85,64267316-22,3.62
2,294400,KOSEF 200TR,2191980,42980,51,2093100,41041.18,4.72,64267316-22,4.25
9,473330,SOL 미국30년국채커버드콜(합성),2067450,9845,210,2040150,9715.00,1.33,64267316-22,4.01
4,357870,TIGER CD금리투자KIS(합성),19631500,55300,355,19617300,55260.00,0.07,64267316-22,38.05
0,241180,TIGER 일본니케이225,1279800,21330,60,1259700,20995.00,1.59,64267316-22,2.48


In [ ]:
total_1

51598237

In [ ]:
df_2

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
2,411060,ACE KRX금현물,3430000,17150,200,3088150,15440.75,11.06,64012548-22
1,360200,ACE 미국S&P500,3270500,21100,155,2825275,18227.58,15.75,64012548-22
0,218420,KODEX 미국S&P500에너지(합성),786500,15730,50,654832,13096.64,20.10,64012548-22
4,473330,SOL 미국30년국채커버드콜(합성),2549855,9845,259,2536490,9793.40,0.52,64012548-22
3,449170,TIGER KOFR금리액티브(합성),2145600,107280,20,2116370,105818.50,1.38,64012548-22
5,0,현금,2054232,1,2054232,2054232,1.00,0.00,64012548-22


In [ ]:
total_2

14236687

In [ ]:
df_3

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
2,411060,ACE KRX금현물,3430000,17150,200,3253647,16268.24,5.42,64043278-01
3,473330,SOL 미국30년국채커버드콜(합성),1870550,9845,190,1856173,9769.33,0.77,64043278-01
0,261120,TIGER 미국달러선물인버스2X,565500,5655,100,563500,5635.00,0.35,64043278-01
1,292560,TIGER 일본엔선물,6686085,8605,777,6681045,8598.51,0.07,64043278-01
4,0,현금,633133,1,633133,633133,1.00,0.00,64043278-01


In [ ]:
total_3

13185268

In [ ]:
df_4["비중"] = df_4["평가금액"] / df_4["평가금액"].sum() * 100
df_4

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌,비중
6,411060,ACE KRX금현물,4081700,17150,238,3917180,16458.74,4.20,64012548-29,11.32
5,365780,ACE 국고채10년,3615300,92700,39,3572105,91592.44,1.21,64012548-29,10.03
4,360200,ACE 미국S&P500,4409900,21100,209,4367815,20898.64,0.96,64012548-29,12.24
1,283580,KODEX 차이나CSI300,1638900,12140,135,1706200,12638.52,-3.94,64012548-29,4.55
2,294400,KOSEF 200TR,1375360,42980,32,1316800,41150.00,4.45,64012548-29,3.82
8,473330,SOL 미국30년국채커버드콜(합성),2953500,9845,300,2922000,9740.00,1.08,64012548-29,8.19
3,357870,TIGER CD금리투자KIS(합성),15373400,55300,278,15362280,55260.00,0.07,64012548-29,42.65
7,456610,TIGER 미국달러SOFR금리액티브(합성),1697100,56570,30,1697355,56578.50,-0.02,64012548-29,4.71
0,241180,TIGER 일본니케이225,895860,21330,42,884335,21055.60,1.30,64012548-29,2.49
9,0,현금,987,1,987,987,1.00,0.00,64012548-29,0.00


In [ ]:
total_4

36042007

In [ ]:
# 그룹화하여 조건에 맞게 계산
result = df.groupby('코드').agg({
    '이름': 'first',
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
    '매입단가': 'mean',
}).reset_index()

result["비중"] = result["평가금액"] / result["평가금액"].sum() * 100
result["수익률"] = (result["평가금액"] - result["매입금액"]) / result["매입금액"] * 100
result.dropna(inplace=True)

# 결과 출력
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률
0,0,현금,3360429,1.00,3360429,3360429,1.00,2.92,0.00
1,218420,KODEX 미국S&P500에너지(합성),786500,15730.00,50,654832,13096.64,0.68,20.11
2,241180,TIGER 일본니케이225,2175660,21330.00,102,2144035,21025.30,1.89,1.48
3,261120,TIGER 미국달러선물인버스2X,565500,5655.00,100,563500,5635.00,0.49,0.35
4,283580,KODEX 차이나CSI300,3508460,12140.00,289,3650660,12632.44,3.05,-3.90
5,292560,TIGER 일본엔선물,6686085,8605.00,777,6681045,8598.51,5.81,0.08
6,294400,KOSEF 200TR,3567340,42980.00,83,3409900,41095.59,3.10,4.62
7,308620,KODEX 미국10년국채선물,4914160,11590.00,424,4875895,11499.75,4.27,0.78
8,357870,TIGER CD금리투자KIS(합성),35004900,55300.00,633,34979580,55260.00,30.42,0.07
9,360200,ACE 미국S&P500,12849900,21100.00,609,12305895,19998.27,11.17,4.42


In [ ]:
eval_total = result['평가금액'].sum()
eval_total

115062199

In [ ]:
# 카테고리 분류
# 114800	KODEX 인버스
# 283580	KODEX 차이나CSI300
# 284430	KODEX 200미국채혼합
# 449170    TIGER KOFR금리액티브(합성)
# 464540    RISE 25-03 회사채(AA-이상)액티브
# 483230    RISE 25-06 은행채(AA+이상)액티브
# 456880    ACE 미국달러SOFR금리(합성)
# 468380    KODEX iShares미국하이일드액티브
# 329750    TIGER 미국달러단기채권액티브
# 261270    KODEX미국달러선물인버스

원화현금 = ["0", "449170", "464540", "483230", "357870"]
달러현금 = ["456880", "468380", "329750", "456610"]
국내주식 = ["396500", "114800", "295040", "294400"]
미국주식 = ["360200", "390390", "402970", "469050", "480020", "482730", "446720", "452360", "489250"]
해외주식 = ["283580", "453810", "241180"]
미국채권 = ["284430", "473330", "308620"]
국내채권 = ["365780" ]
대채투자 = ["411060", "475380", "218420"]
달러헷지 = ["261270", "476750", "261120", "292560"]

result['카테고리'] = '기타'
result.loc[result['코드'].isin(원화현금), '카테고리'] = '원화현금'
result.loc[result['코드'].isin(달러현금), '카테고리'] = '달러현금'
result.loc[result['코드'].isin(국내주식), '카테고리'] = '국내주식'
result.loc[result['코드'].isin(미국주식), '카테고리'] = '미국주식'
result.loc[result['코드'].isin(해외주식), '카테고리'] = '해외주식'
result.loc[result['코드'].isin(미국채권), '카테고리'] = '미국채권'
result.loc[result['코드'].isin(국내채권), '카테고리'] = '국내채권'
result.loc[result['코드'].isin(대채투자), '카테고리'] = '대채투자'
result.loc[result['코드'].isin(달러헷지), '카테고리'] = '달러헷지'
result.sort_values(by='카테고리', inplace=True)
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리
6,294400,KOSEF 200TR,3567340,42980.00,83,3409900,41095.59,3.10,4.62,국내주식
10,365780,ACE 국고채10년,7879500,92700.00,85,7785255,91591.33,6.85,1.21,국내채권
3,261120,TIGER 미국달러선물인버스2X,565500,5655.00,100,563500,5635.00,0.49,0.35,달러헷지
5,292560,TIGER 일본엔선물,6686085,8605.00,777,6681045,8598.51,5.81,0.08,달러헷지
13,456610,TIGER 미국달러SOFR금리액티브(합성),1697100,56570.00,30,1697355,56578.50,1.47,-0.02,달러현금
14,456880,ACE 미국달러SOFR금리(합성),4393010,11470.00,383,4365560,11398.33,3.82,0.63,달러현금
1,218420,KODEX 미국S&P500에너지(합성),786500,15730.00,50,654832,13096.64,0.68,20.11,대채투자
11,411060,ACE KRX금현물,16086700,17150.00,938,15192290,16153.03,13.98,5.89,대채투자
9,360200,ACE 미국S&P500,12849900,21100.00,609,12305895,19998.27,11.17,4.42,미국주식
7,308620,KODEX 미국10년국채선물,4914160,11590.00,424,4875895,11499.75,4.27,0.78,미국채권


In [ ]:
# 새로운 '분류' 컬럼 추가
result['대분류'] = result['카테고리']
result.loc[result['카테고리'].str.contains('현금'), '대분류'] = '현금'
result.loc[result['카테고리'].str.contains('주식'), '대분류'] = '주식'
result.loc[result['카테고리'].str.contains('채권'), '대분류'] = '채권'
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
6,294400,KOSEF 200TR,3567340,42980.00,83,3409900,41095.59,3.10,4.62,국내주식,주식
10,365780,ACE 국고채10년,7879500,92700.00,85,7785255,91591.33,6.85,1.21,국내채권,채권
3,261120,TIGER 미국달러선물인버스2X,565500,5655.00,100,563500,5635.00,0.49,0.35,달러헷지,달러헷지
5,292560,TIGER 일본엔선물,6686085,8605.00,777,6681045,8598.51,5.81,0.08,달러헷지,달러헷지
13,456610,TIGER 미국달러SOFR금리액티브(합성),1697100,56570.00,30,1697355,56578.50,1.47,-0.02,달러현금,현금
14,456880,ACE 미국달러SOFR금리(합성),4393010,11470.00,383,4365560,11398.33,3.82,0.63,달러현금,현금
1,218420,KODEX 미국S&P500에너지(합성),786500,15730.00,50,654832,13096.64,0.68,20.11,대채투자,대채투자
11,411060,ACE KRX금현물,16086700,17150.00,938,15192290,16153.03,13.98,5.89,대채투자,대채투자
9,360200,ACE 미국S&P500,12849900,21100.00,609,12305895,19998.27,11.17,4.42,미국주식,주식
7,308620,KODEX 미국10년국채선물,4914160,11590.00,424,4875895,11499.75,4.27,0.78,미국채권,채권


In [ ]:
# 카테고리 별 비중 계산
대분류 = result.groupby('대분류').agg({
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
}).reset_index()

total_amount = 대분류['평가금액'].sum()
대분류['비중'] = 대분류['평가금액'] / total_amount * 100
대분류["수익률"] = (대분류["평가금액"] - 대분류["매입금액"]) / 대분류["매입금액"] * 100
대분류

,대분류,평가금액,단가,수량,매입금액,비중,수익률
0,달러헷지,7251585,7130.00,877,7244545,6.30,0.10
1,대채투자,16873200,16440.00,988,15847122,14.66,6.47
2,주식,22101360,24387.50,1083,21510490,19.21,2.75
3,채권,22235015,38045.00,1468,22015963,19.32,0.99
4,현금,46601039,46124.20,3361495,46519294,40.50,0.18


In [ ]:
# 카테고리 별 비중 계산
category_group = result.groupby('카테고리').agg({
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
}).reset_index()

total_amount = category_group['평가금액'].sum()
category_group['비중'] = category_group['평가금액'] / total_amount * 100
category_group["수익률"] = (category_group["평가금액"] - category_group["매입금액"]) / category_group["매입금액"] * 100
category_group

,카테고리,평가금액,단가,수량,매입금액,비중,수익률
0,국내주식,3567340,42980.00,83,3409900,3.10,4.62
1,국내채권,7879500,92700.00,85,7785255,6.85,1.21
2,달러헷지,7251585,7130.00,877,7244545,6.30,0.10
3,달러현금,6090110,34020.00,413,6062915,5.29,0.45
4,대채투자,16873200,16440.00,988,15847122,14.66,6.47
5,미국주식,12849900,21100.00,609,12305895,11.17,4.42
6,미국채권,14355515,10717.50,1383,14230708,12.48,0.88
7,원화현금,40510929,54193.67,3361082,40456379,35.21,0.13
8,해외주식,5684120,16735.00,391,5794695,4.94,-1.91


In [ ]:
# '대분류'가 '채권'인 항목 필터링
# 달러_df = result[result['카테고리'] == '대채투자' | 
#                  (result['카테고리'] == '미국주식') | 
#                  (result['카테고리'] == '해외주식') |
#                  (result['카테고리'] == '미국채권') ]

달러_df = pd.concat([result[result['카테고리'] == '미국주식'],
                    result[result['카테고리'] == '달러현금'],
                    result[result['카테고리'] == '미국채권']])

달러인버스 = ["261270", 
         "476750", 
         "411060", 
         "261120", 
         "241180", 
         "283580"]
달러헷지_df = result.loc[result['코드'].isin(달러인버스)]

# 총 평가금액 계산
total_달러_amount = 달러_df['평가금액'].sum()
total_달러헷지_amount = 달러헷지_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
달러_df.loc[:, '비중'] = 달러_df['평가금액'] / total_달러_amount * 100
달러헷지_df.loc[:, '비중'] = 달러헷지_df['평가금액'] / total_달러헷지_amount * 100
display(달러_df)
display(달러헷지_df)
display(total_달러_amount)
display(total_달러헷지_amount)
display(total_달러_amount - total_달러헷지_amount)

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
9,360200,ACE 미국S&P500,12849900,21100.00,609,12305895,19998.27,38.59,4.42,미국주식,주식
13,456610,TIGER 미국달러SOFR금리액티브(합성),1697100,56570.00,30,1697355,56578.50,5.10,-0.02,달러현금,현금
14,456880,ACE 미국달러SOFR금리(합성),4393010,11470.00,383,4365560,11398.33,13.19,0.63,달러현금,현금
7,308620,KODEX 미국10년국채선물,4914160,11590.00,424,4875895,11499.75,14.76,0.78,미국채권,채권
15,473330,SOL 미국30년국채커버드콜(합성),9441355,9845.00,959,9354813,9754.43,28.36,0.93,미국채권,채권


,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
3,261120,TIGER 미국달러선물인버스2X,565500,5655.00,100,563500,5635.00,2.53,0.35,달러헷지,달러헷지
11,411060,ACE KRX금현물,16086700,17150.00,938,15192290,16153.03,72.02,5.89,대채투자,대채투자
2,241180,TIGER 일본니케이225,2175660,21330.00,102,2144035,21025.30,9.74,1.48,해외주식,주식
4,283580,KODEX 차이나CSI300,3508460,12140.00,289,3650660,12632.44,15.71,-3.90,해외주식,주식


33295525

22336320

10959205

In [ ]:
# '대분류'가 '채권'인 항목 필터링
bond_df = result[result['대분류'] == '채권']

# 총 평가금액 계산
total_bond_amount = bond_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
bond_df.loc[:, '비중'] = bond_df['평가금액'] / total_bond_amount * 100
bond_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
10,365780,ACE 국고채10년,7879500,92700.00,85,7785255,91591.33,35.44,1.21,국내채권,채권
7,308620,KODEX 미국10년국채선물,4914160,11590.00,424,4875895,11499.75,22.10,0.78,미국채권,채권
15,473330,SOL 미국30년국채커버드콜(합성),9441355,9845.00,959,9354813,9754.43,42.46,0.93,미국채권,채권


In [ ]:
# '대분류'가 '주식'인 항목 필터링
stock_df = result[result['대분류'] == '주식']

# 총 평가금액 계산
total_stock_amount = stock_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
stock_df.loc[:, '비중'] = stock_df['평가금액'] / total_stock_amount * 100
stock_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
6,294400,KOSEF 200TR,3567340,42980.00,83,3409900,41095.59,16.14,4.62,국내주식,주식
9,360200,ACE 미국S&P500,12849900,21100.00,609,12305895,19998.27,58.14,4.42,미국주식,주식
2,241180,TIGER 일본니케이225,2175660,21330.00,102,2144035,21025.30,9.84,1.48,해외주식,주식
4,283580,KODEX 차이나CSI300,3508460,12140.00,289,3650660,12632.44,15.87,-3.90,해외주식,주식


In [ ]:
# '대분류'가 '현금'인 항목 필터링
cash_df = result[result['대분류'] == '현금']

# 총 평가금액 계산
total_cash_amount = cash_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
cash_df.loc[:, '비중'] = cash_df['평가금액'] / total_cash_amount * 100
cash_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
13,456610,TIGER 미국달러SOFR금리액티브(합성),1697100,56570.00,30,1697355,56578.50,3.64,-0.02,달러현금,현금
14,456880,ACE 미국달러SOFR금리(합성),4393010,11470.00,383,4365560,11398.33,9.43,0.63,달러현금,현금
0,0,현금,3360429,1.00,3360429,3360429,1.00,7.21,0.00,원화현금,현금
8,357870,TIGER CD금리투자KIS(합성),35004900,55300.00,633,34979580,55260.00,75.12,0.07,원화현금,현금
12,449170,TIGER KOFR금리액티브(합성),2145600,107280.00,20,2116370,105818.50,4.60,1.38,원화현금,현금



## 최근 영업일

In [ ]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2024.11.21


In [ ]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20241121
